# Session 17: Introduction to Linear Programming (LP)

## Example: Production Planning

**(DMD Ex. 7.2)** The Gemstone Tool Company (GTC) produces wrenches and pliers. Each product is made of steel, and requires using a Molding Machine and an Assembly Machine. The daily availability of each resource, as well as the resources required to produce one units of each tool, are shown below.

| ` `| Wrench (1 unit) | Plier (1 unit) | Daily Availability |
|--|--|--|--|
|Steel | 1.5 lbs | 1.0 lbs | 27,000 lbs|
|Molding Machine | 1.0 hours | 1.0 hours | 21,000 hours |
|Assembly Machine| 0.3 hours | 0.5 hours | 9,000 hours |

There is demand for 16,000 wrenches and 15,000 pliers per day. Each wrench earns a profit of .10 dollars and each plier earns a profit of .13 dollars. 

**a)** How much of each product should GTC produce each day and what is the maximum possible profit?

**b)** How much additional profit can the company obtain if it had one additional unit of each of the three resources?

### Step 1. Identify the decision, objective, and constraints in English

**Decision:** 

**Objective:**

**Constraints:** 


### Step 2. Formulate the optimization as linear expressions of decision variables

**Decision Variables:** 


**Objective:**

**Constraints:**

### Step 3. Numerically solve using Gurobi

In [1]:
import gurobipy as grb
mod=grb.Model()
W=mod.addVar()
P=mod.addVar()
mod.setObjective(.1*W+.13*P,sense=grb.GRB.MAXIMIZE)
steel=mod.addConstr(1.5*W+P <= 27000)
molding=mod.addConstr(W+P <=21000)
assembly=mod.addConstr(.3*W+.5*P<=9000)
mod.addConstr(W<=16000)
mod.addConstr(P<=15000)
mod.optimize()

Academic license - for non-commercial use only
Optimize a model with 5 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+03, 3e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5100000e+03   3.375000e+03   0.000000e+00      0s
       3    2.5050000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  2.505000000e+03


In [2]:
print('Optimal profit:',mod.objval)
print('W:',W.x)
print('P:',P.x)
print('\nShadow prices:')
print(f'Steel {steel.pi} \t valid RHS: {steel.sarhslow} to {steel.sarhsup}')
print(f'Molding {molding.pi :.3f} \t valid RHS: {molding.sarhslow} to {molding.sarhsup}')
print(f'Assembly {assembly.pi :.3f} \t valid RHS: {assembly.sarhslow} to {assembly.sarhsup}')

Optimal profit: 2505.0
W: 7500.0
P: 13500.0

Shadow prices:
Steel 0.0 	 valid RHS: 24750.0 to 1e+100
Molding 0.055 	 valid RHS: 20000.0 to 22000.0
Assembly 0.150 	 valid RHS: 8100.0 to 9300.0


### Debugging by Outputing Formulation

In [3]:
mod.write('GTC.lp')
!cat GTC.lp

\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.1 C0 + 0.13 C1
Subject To
 R0: 1.5 C0 + C1 <= 27000
 R1: C0 + C1 <= 21000
 R2: 0.3 C0 + 0.5 C1 <= 9000
 R3: C0 <= 16000
 R4: C1 <= 15000
Bounds
End


In [11]:
# Naming variables and constraints
import gurobipy as grb
mod=grb.Model()
W=mod.addVar(name='W')
P=mod.addVar(name='P')
mod.setObjective(.1*W+.13*P,sense=grb.GRB.MAXIMIZE)
steel=mod.addConstr(1.5*W+P <= 27000,name='Steel')
molding=mod.addConstr(W+P <=21000,name='Molding')
assembly=mod.addConstr(.3*W+.5*P<=9000,name='Assembly')
mod.addConstr(W<=16000,name='Demand-W')
mod.addConstr(P<=15000,name='Demand-P')
mod.write('GTC.lp')
!cat GTC.lp

\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.1 W + 0.13 P
Subject To
 Steel: 1.5 W + P <= 27000
 Molding: W + P <= 21000
 Assembly: 0.3 W + 0.5 P <= 9000
 Demand-W: W <= 16000
 Demand-P: P <= 15000
Bounds
End


## Exercise: Transportation Planning

There are 2 production plants, A and B, with capacities $20$ and $15$ respectively. There are 3 demand centers, 1, 2, 3, with demand of $10$ each. The cost of transporting each unit of good from each plant to each demand center is shown below.

|` ` |1 | 2|3|
|--|--|--|--|
|A|3|7|5|
|B|5|3|3|

**a)** What is the minimum transportation cost needed to satisfy all demand while respecting plant capacities, and how would you achieve this cost? 

**b)** How would increasing one unit of capacity at each plant affect the optimal cost?

**c)** How would increasing one unit of demand at each center affect the optimal cost?

### Step 1. Identify the decision, objective, and constraints in English

**Decision:** 

**Objective:**

**Constraints:**


### Step 2. Formulate the optimization as linear expressions of decision variables

**Decision variables:** 

**Objective and Constraints:** 

### Step 3. Numerically solve using Gurobi

A) Minimal cost: 100.0
Optimal transportation plan:
	A1: 10.0
	A2: 0.0
	A3: 5.0
	B1: 0.0
	B2: 10.0
	B3: 5.0


B) Effect of adding 1 unit of plant capacity
	 Plant A: 0.0 	Valid RHS: 15.0 to 1e+100
	 Plant B: -2.0 	Valid RHS: 10.0 to 20.0


C) Effect of demand increase by 1 unit
	 Center 1: 3.0 	Valid RHS: -0.0 to 15.0
	 Center 2: 5.0 	Valid RHS: 5.0 to 15.0
	 Center 3: 5.0 	Valid RHS: 5.0 to 15.0
